# 题目
Write a script to compute the decision tree for an input dataset.

You can assume that all attributes are numeric, except for the last attribute which is the class.

Use the Information Gain based on Entropy for computing the best split value for eachattribute. For the stopping criteria at a node, stop if the purity is at least 95% or stop if thenode size is five or lower.

Output your decision tree. Note that each internal node, print the decision followed by thenformation Gain, and for each leaf, print the majority label, purity of the leaf, and the size.

# 决策树
决策树模型是运用于分类以及回归的一种树结构。决策树由节点和有向边组成，一般一棵决策树包含一个根节点、若干内部节点和若干叶节点。决策树的决策过程需要从决策树的根节点开始，待测数据与决策树中的特征节点进行比较，并按照比较结果选择选择下一比较分支，直到叶子节点作为最终的决策结果。

目标变量可以采用一组离散值的树模型称为分类树(常用的分类树算法有ID3、C4.5、CART)，而目标变量可以采用连续值（通常是实数）的决策树被称为回归树(如CART算法)。

决策树算法本质上就是要找出每一列的最佳划分以及不同列划分的先后顺序及排布。

信息熵表示的是不确定度。均匀分布时，不确定度最大，此时熵就最大。当选择某个特征对数据集进行分类时，分类后的数据集信息熵会比分类前的小，其差值表示为信息增益。信息增益可以衡量某个特征对分类结果的影响大小。

假设在样本数据集 D 中，混有 c 种类别的数据。构建决策树时，根据给定的样本数据集选择某个特征值作为树的节点。在数据集中，可以计算出该数据中的信息熵：![Image of Yaktocat](https://www.ibm.com/developerworks/cn/analytics/library/ba-1507-decisiontree-algorithm/img02.png)
对应数据集 D，选择特征 A 作为决策树判断节点时，在特征 A 作用后的信息熵的为 Info(D)，计算如下：![Image of Yaktocat](https://www.ibm.com/developerworks/cn/analytics/library/ba-1507-decisiontree-algorithm/img03.png)
信息增益表示数据集 D 在特征 A 的作用后，其信息熵减少的值。公式如下：![Image of Yaktocat](https://www.ibm.com/developerworks/cn/analytics/library/ba-1507-decisiontree-algorithm/img04.png)
对于决策树节点最合适的特征选择，就是 Gain(A) 值最大的特征。

在分类模型建立的过程中，很容易出现过拟合的现象。过拟合是指在模型学习训练中，训练样本达到非常高的逼近精度，但对检验样本的逼近误差随着训练次数而呈现出先下降后上升的现象。过拟合时训练误差很小，但是检验误差很大，不利于实际应用。

决策树的过拟合现象可以通过剪枝进行一定的修复。剪枝分为预先剪枝和后剪枝两种。

预先剪枝指在决策树生长过程中，使用一定条件加以限制，使得产生完全拟合的决策树之前就停止生长。预先剪枝的判断方法也有很多，比如信息增益小于一定阀值的时候通过剪枝使决策树停止生长。但如何确定一个合适的阀值也需要一定的依据，阀值太高导致模型拟合不足，阀值太低又导致模型过拟合。

后剪枝是在决策树生长完成之后，按照自底向上的方式修剪决策树。后剪枝有两种方式，一种用新的叶子节点替换子树，该节点的预测类由子树数据集中的多数类决定。另一种用子树中最常使用的分支代替子树。

预先剪枝可能过早的终止决策树的生长，后剪枝一般能够产生更好的效果。但后剪枝在子树被剪掉后，决策树生长的一部分计算就被浪费了。

In [32]:
import numpy as np
import pandas as pd
import math
filepath = "C:\\Users\\Shinelon\\Desktop\\data-mining  project\\Project4\\iris.txt"
df = pd.read_csv(filepath).values
df=df[:,[0,1,4]]

df[df[:,2]=='Iris-setosa',2]=1
df[df[:,2]=='Iris-versicolor',2]=2
df[df[:,2]=='Iris-virginica',2]=2
# print(df)

#计算某一属性里面的能取得最大Gain的对应的split_point，以及其gain
def evaluate_numeric_attribute(D,attribute,sigma):
    featureList = [instance[attribute] for instance in D]
    uniqueVals = set(featureList)
    uniqueVals=sorted(list(uniqueVals))
    split_point=0.0
    BestGain=0.0
    for i in range(len(uniqueVals)-1):
        v=(uniqueVals[i]+uniqueVals[i+1])/2.0
        DY,DN=splitPointDataset(D,attribute,v)
        if len(DY)<=sigma or len(DN)<=sigma:
            continue
        baseEnt = calEnt(D)
        Gain=baseEnt-float(len(DY))/len(D)*calEnt((DY))-float(len(DN))/len(D)*calEnt(DN)
        if Gain>BestGain:
            BestGain=Gain
            split_point=v
    return split_point,BestGain


#计算数据的类别属性里面比例最大的类别的label以及其纯度
def majorityLabel_purity(classList):
    if len(classList)==0:
        return None,0.0,0
    classCount={}
    for n in classList:
        if n not in classCount.keys():
            classCount[n]=0
        classCount[n]+=1
    #print classCount
    sortedCC=sorted(classCount.items(),key=lambda x:x[1],reverse=True)
    majorityLabel=sortedCC[0][0]
    purity=1.0*sortedCC[0][1]/len(classList)
    num=len(classList)
    return [majorityLabel,purity,num]

#熵
def calEnt(D):
    num=len(D)
    labelCount={}
    for instance in D:
        currentLabel=instance[-1]
        if currentLabel not in labelCount.keys():
            labelCount[currentLabel]=0
        labelCount[currentLabel]+=1
    Ent=0.0
    for key in labelCount:
        prob=float(labelCount[key])/num
        Ent-=prob*math.log(prob,2)
    return Ent

#通过split_point对数据进行split处理
def splitPointDataset(D,attribute,value):
    DY=[]
    DN=[]
    for instance in D:
        if instance[attribute]<=value:
            DY.append(instance)
        if instance[attribute]>value:
            DN.append(instance)
    return DY,DN

#选择最优的属性
def chooseBestFeature(dataSet,sigma,pai):
    classList = [instance[-1] for instance in dataSet]
    majorityLabel,purity,num=majorityLabel_purity(classList)
    if len(set(classList))==1:     # 停止条件 1
        return None,majorityLabel,purity,num               # 返回标签的多数类别作为叶子节点
    if purity>=pai:               # 停止条件 2   如果所给的数据纯度达到标准，则停止划分
        return None,majorityLabel,purity,num
    featnum=len(dataSet[0])-1
    bestFeature=-1
    bestGain=0.0
    bestSplit_point=0.0
    for i in range(featnum):
        split_point,Gain = evaluate_numeric_attribute(dataSet,i,sigma)
        if Gain>bestGain:
            bestGain=Gain
            bestFeature=i
            bestSplit_point=split_point
    dY, dN = splitPointDataset(dataSet,bestFeature,bestSplit_point)
    if len(dY) < sigma or len(dN) < sigma:        # 停止条件 3
        return None,majorityLabel,purity,num
    return bestFeature,bestSplit_point,bestGain,num

#构造决策树
def createTree(D,sigma,pai):
    bestFeature, split_point,purity,num= chooseBestFeature(D,sigma,pai)
    if bestFeature == None: return {'label':split_point,'purity':purity,'number':num}
    DY, DN = splitPointDataset(D, bestFeature, split_point)
    Decision_tree = {}
    Decision_tree['attribute'] = bestFeature
    Decision_tree['split_point'] = split_point
    Decision_tree['Gain']=purity
    Decision_tree['left'] = createTree(DY, sigma, pai)
    Decision_tree['right'] = createTree(DN, sigma, pai)
    return Decision_tree


df=df.tolist()
myTree = createTree(df,2,0.95)
print(myTree)

{'attribute': 0, 'split_point': 5.45, 'Gain': 0.5309046836085699, 'left': {'attribute': 1, 'split_point': 2.8, 'Gain': 0.35726699979283877, 'left': {'attribute': 0, 'split_point': 4.95, 'Gain': 0.19811742113040343, 'left': {'label': 2, 'purity': 0.6666666666666666, 'number': 3}, 'right': {'label': 2, 'purity': 1.0, 'number': 4}}, 'right': {'label': 1, 'purity': 0.9777777777777777, 'number': 45}}, 'right': {'attribute': 1, 'split_point': 3.45, 'Gain': 0.2142137447120585, 'left': {'label': 2, 'purity': 1.0, 'number': 89}, 'right': {'attribute': 0, 'split_point': 6.5, 'Gain': 0.9544340029249649, 'left': {'label': 1, 'purity': 1.0, 'number': 5}, 'right': {'label': 2, 'purity': 1.0, 'number': 3}}}}


# 参考
https://github.com/nanzai9996/kernel_trick-and_denclue_and_decison_tree-/blob/master/decision_tree.py
https://blog.csdn.net/weixin_36586536/article/details/80468426